In [ ]:
import numpy as np
import pandas as pd 
import os
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

basedir = '.'

Se define una funcion que obtiene una hoja especifica de todos los anuarios 2013-2020 y se almacenan en una lista

In [5]:

def get_sets_from_xlsx(file_dir,sheet_num, years=np.arange(2013,2021) ) -> list:
    ''' Esta funcion  devuelve una lista de DataFrames de la misma hija de excel de cada anuario 
    filedir: directorio de los anuarios
    sheet_num: numero de la hoja que se desea extraer
    years: años que se desean analizar, el default es todos'''

    sets_list = []

    for year in years:
        
        filepath = file_dir + f'\data_{year}.xlsx'
        file = pd.ExcelFile(filepath)
        
        sheet_name = file.sheet_names[sheet_num]
        
        df = pd.read_excel(filepath,sheet_name=sheet_name,header=None,dtype=str)

        df['Año'] = year

        sets_list.append(df)
    
    return sets_list




In [ ]:

raw_data_dir = os.path.join(basedir,'raw_data')

# extraer info de la hoja numero 2 '1.1.1 Establecimientos de hospedaje'

data_sets = get_sets_from_xlsx(raw_data_dir,1)

# Sobrescribir el anuario 2013 en la hoja correcta, porque tiene una que recorre todas las hojas portada y no se puede modificar 

data_sets[0] = get_sets_from_xlsx(raw_data_dir,2)[0]


In [7]:
data_sets

[         0                                                  1  \
 0      NaN                                                NaN   
 1    1.1.1  Establecimientos de hospedaje registrados por ...   
 2      NaN      según categoría turística del establecimiento   
 3      NaN                            Región\n      Municipio   
 4      NaN                                                NaN   
 ..     ...                                                ...   
 132    NaN                                                NaN   
 133    NaN                                                  *   
 134    NaN                                                 **   
 135    NaN                                                NaN   
 136    NaN                                            Fuente:   
 
                                                      2    3    4      5  \
 0                                                  NaN  NaN  NaN    NaN   
 1                                                  Na

Veamos como vienen los datos

In [8]:
df1 = data_sets[3]
df1

,0,1,2,3,4,5,6,7,8,9,10,Año
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
2,NaN,1.1.1 Establecimientos de hospedaje registrado...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
3,NaN,según categoría turística del establecimiento,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
...,...,...,...,...,...,...,...,...,...,...,...,...
139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
140,NaN,*,Incluye establecimientos de categoría especial...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
141,NaN,**,"Comprende: hoteles, moteles, casas de huéspede...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016


In [ ]:
def filter_dataframe(df,columns):
    """
     Filter and clean DataFrame based on hardcoded criteria.
    Args:
        df: pandas DataFrame to process
        columns: list of column names to assign
    """


    # Obtener el indice de la fila apartir de la de cual  empezara el DataFrame  
     first_row_idx = np.where((df == 'Estado') | (df == 'Municipio').any(axis=1))[0][0]
    
    # Obtener la columna apartir de la cual empezara el DataFrame, la que contiene los nombres de municipios
     _,col_idx = ((df == 'Guadalajara') | (df == 'GUADALAJARA')).values.nonzero()
     col_idx = col_idx[0]


    # Obtener la columna en la que termina el DataFrame, la que es igual a Zapotlanejo
     last_row_idx,_ = ((df == 'Zapotlanejo') | (df == 'ZAPOTLANEJO')).values.nonzero() 
     last_row_idx = last_row_idx[0]

    # Redefinir el df en el rango deseado
     df = df.iloc[first_row_idx:last_row_idx + 1,col_idx:]

    # Dropear filas que quedan con nulos
     df = df.dropna(axis='columns',how='all')
     df = df.dropna(axis='rows',how='any')
     
     # redifinir las columnas
     df.columns = columns     


     return df
     

In [12]:
# Test de la funcion

columns = ['municipio','total','5 estrellas','4 estrellas','3 estrellas','2 estrellas','1 estrellas','sin categoria','año']
df2 = filter_dataframe(data_sets[1],columns)
df2

,municipio,total,5 estrellas,4 estrellas,3 estrellas,2 estrellas,1 estrellas,sin categoria,año
9,Bolaños,3,0,0,0,0,1,2,2014
10,Chimaltitán,1,0,0,0,0,0,1,2014
11,Colotlán,2,0,1,0,0,1,0,2014
12,Huejúcar,4,0,0,0,0,4,0,2014
13,Huejuquilla el Alto,4,0,0,2,0,0,2,2014
...,...,...,...,...,...,...,...,...,...
128,Tlajomulco de Zúñiga,7,1,1,1,0,0,4,2014
129,Tonalá,11,0,0,6,0,3,2,2014
130,Villa Corona,10,0,0,0,0,2,8,2014
131,Zapopan,70,8,19,11,2,1,29,2014


In [ ]:
# Aplicarla a cada data set
cleaned_sets = []
columns = ['municipio','total','5 estrellas','4 estrellas','3 estrellas','2 estrellas','1 estrellas','sin categoria','año']


for df in data_sets:
    clean_df = filter_dataframe(df,columns)
    cleaned_sets.append(clean_df)


a
a
a
a
a
a
a
a


In [14]:
cleaned_sets

[                municipio total 5 estrellas 4 estrellas 3 estrellas  \
 6                  Estado  1647         106         204         273   
 7                 I Norte    28           0           2           2   
 8                 Bolaños     3           0           0           0   
 9             Chimaltitán     1           0           0           0   
 10               Colotlán     2           0           1           0   
 ..                    ...   ...         ...         ...         ...   
 127  Tlajomulco de Zúñiga     7           1           1           1   
 128                Tonalá    11           0           0           6   
 129          Villa Corona    10           0           0           0   
 130               Zapopan    70           8          19          11   
 131           Zapotlanejo     7           0           1           3   
 
     2 estrellas 1 estrellas sin categoria   año  
 6           148         230           685  2013  
 7             1           8    

In [28]:
# guardar todos los sets en su direccion correspondiente

final_dir = os.path.join(basedir,r'data\Hoteles')
final_dir
for i,df in enumerate(cleaned_sets):
    df.to_csv(f'{final_dir}_{2013+i}.csv')
